In [1]:
pip install keras-tuner --upgrade

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.8.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
fashion_mnist

<module 'keras.api._v2.keras.datasets.fashion_mnist' from '/usr/local/lib/python3.7/dist-packages/keras/api/_v2/keras/datasets/fashion_mnist/__init__.py'>

In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [7]:
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images.shape

(60000, 28, 28)

In [10]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [11]:
#hp is hyperparameter from keras-tuner
def build_model(hp):  
  #NN 
  model = keras.Sequential([ 
    keras.layers.Conv2D( 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16), # number of output filters in the convolution
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]), #list of 2 integers, specifying the height and width of the 2D convolution window.
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),#convert input dimentions into 1D dimentions for ann to read
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    #softmax, in terms of percentages
    keras.layers.Dense(10, activation='softmax') #output nodes, softmax will give probability of class
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy', #multiclass determination
              metrics=['accuracy'])
  
  return model

In [12]:
pip install keras-tuner

In [13]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [15]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',
                          project_name='fashion_mnist')

In [16]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 01m 22s]
val_accuracy: 0.9013333320617676

Best val_accuracy So Far: 0.906333327293396
Total elapsed time: 00h 06m 09s
INFO:tensorflow:Oracle triggered exit
